In [19]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import seaborn as sns
import json
import datetime
data_df = pd.read_json('C:/Users/spatan5/Desktop/INMAS/Final project/IMDB_movie_details.json', lines=True)
#data_rev = pd.read_json('C:/Users/spatan5/Desktop/INMAS/Final project/IMDB_reviews.json', lines=True)

#print(data_rev.head())

data_df.info()

train = data_df[0: 1200]

test = data_df[1200 : ]

 
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       1572 non-null   object 
 1   plot_summary   1572 non-null   object 
 2   duration       1572 non-null   object 
 3   genre          1572 non-null   object 
 4   rating         1572 non-null   float64
 5   release_date   1572 non-null   object 
 6   plot_synopsis  1572 non-null   object 
dtypes: float64(1), object(6)
memory usage: 86.1+ KB
    movie_id                                       plot_summary  duration  \
0  tt0105112  Former CIA analyst, Jack Ryan is in England wi...  1h 57min   
1  tt1204975  Billy (Michael Douglas), Paddy (Robert De Niro...  1h 45min   
2  tt0243655  The setting is Camp Firewood, the year 1981. I...  1h 37min   
3  tt0040897  Fred C. Dobbs and Bob Curtin, both down on the...   2h 6min   
4  tt0126886  Tracy Flick is running unopposed for this year

In [20]:
from time import time
from optparse import OptionParser
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
op = OptionParser()
argv = []
sys.argv[1:]
(opts, args) = op.parse_args(argv)

In [35]:
print("Extracting features from the training data using a count vectorizer")
t0 = time()
countvec = CountVectorizer(stop_words='english',max_features=2**14, max_df=0.5, min_df=0.05,)
X_train = countvec.fit_transform(train['plot_summary'])
X_test  = countvec.transform(test['plot_summary']) # Extracting features from the test data using the same vectorizer
duration = time() - t0
# check computational effort to compute the features
#print("done in %fs at %0.3fMB/s" % (duration, size_mb(train) / duration))
#print("n_samples: %d, n_features: %d" % X_train.shape)

Extracting features from the training data using a count vectorizer


In [16]:
import pandas as pd
X_train_countvec_df = pd.DataFrame(X_train.todense())

# This are the different "words" that are in our vocabulary:
X_train_countvec_df.columns = sorted(countvec.vocabulary_)
print(X_train_countvec_df.columns)
# This shows how a rows of our feature matrix look like:
X_train_countvec_df

Index(['duration', 'genre', 'movie_id', 'plot_summary', 'plot_synopsis',
       'rating', 'release_date'],
      dtype='object')


,duration,genre,movie_id,plot_summary,plot_synopsis,rating,release_date
0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0
5,0,0,0,0,0,0,1
6,0,0,0,0,1,0,0


In [32]:
print("Extracting features from the training data using a TF-IDF vectorizer")
t0 = time()
vectorizer_tfidf = TfidfVectorizer(sublinear_tf=True, max_df=0.5, min_df=0.05,
                             stop_words='english')
X_train_tfidf = vectorizer_tfidf.fit_transform(train)
X_test_tfidf = vectorizer_tfidf.transform(data_df) # Extracting features from the test data using the same vectorizer
duration = time() - t0
#print("done in %fs at %0.3fMB/s" % (duration, size_mb(train)  / duration))
#print("n_features: %d" % X_train_tfidf.shape[1])
print()

Extracting features from the training data using a TF-IDF vectorizer



In [36]:
X_train.shape

(1200, 80)

In [37]:
y_train = train['rating'].astype(str)
y_test= test['rating'].astype(str)

from sklearn.linear_model import LogisticRegression
t0 = time()
lr =LogisticRegression(solver='lbfgs',max_iter=150,multi_class='multinomial').fit(X_train,y_train)
print("Runtime of training of "+str(lr)+" with count vectorizer encoding: ",format(time()-t0,"0.3f"),"s")
t0 = time()
print("Mean accuracy of model "+str(lr)+" on training data with count vectorizer encoding: ",lr.score(X_train,y_train))
print("Mean accuracy of model "+str(lr)+" on test data with count vectorizer encoding: ",lr.score(X_test,y_test))
print("Runtime of evaluating "+str(lr)+" on training and test data with count vectorizer encoding: ",format(time()-t0,"0.3f"),"s")


# We observe that the training time is reasonable, but not trivial anymore. On the other hand, evaluating the model (calculating the accuaracy) is still very quick
# 
# Now, we repeat this with the TF-IDF features.

Runtime of training of LogisticRegression(max_iter=150, multi_class='multinomial') with count vectorizer encoding:  0.195 s
Mean accuracy of model LogisticRegression(max_iter=150, multi_class='multinomial') on training data with count vectorizer encoding:  0.345
Mean accuracy of model LogisticRegression(max_iter=150, multi_class='multinomial') on test data with count vectorizer encoding:  0.04032258064516129
Runtime of evaluating LogisticRegression(max_iter=150, multi_class='multinomial') on training and test data with count vectorizer encoding:  0.000 s


In [29]:
t0 = time()
lr2 =LogisticRegression(solver='lbfgs',max_iter=150,multi_class='multinomial').fit(X_train_tfidf,y_train)
print("Runtime of training of "+str(lr2)+" with TF-IDF encoding: ",format(time()-t0,"0.3f"),"s")
t0 = time()
print("Mean accuracy of model "+str(lr2)+" on training data with TF-IDF encoding: ",lr2.score(X_train_tfidf,y_train))
print("Mean accuracy of model "+str(lr2)+" on test data with TF-IDF encoding: ",lr2.score(X_test_tfidf,y_test))
print("Runtime of evaluating "+str(lr2)+" on training and test data with TF-IDF encoding: ",format(time()-t0,"0.3f"),"s")

text_test = ['Bill Gates and Steve Jobs are computer enterpreneurs.','Elon Musk wants to fly to Mars.']
X_custom_counts=countvec.transform(text_test)
X_custom_tfidf=vectorizer_tfidf.transform(text_test)

predicted=lr.predict(X_custom_counts)
predicted2=lr2.predict(X_custom_tfidf)

print("Predicuts of model "+str(lr)+" (count vectorizer encoding)")
for doc,category in zip(text_test,predicted):
    print('%r => %s'%(doc,train.target_names[category]))
print("Predicuts of model "+str(lr2)+" (TF-IDF encoding)")
for doc,category in zip(text_test,predicted2):
    print('%r => %s'%(doc,train.target_names[category]))

ValueError: Found input variables with inconsistent numbers of samples: [7, 1200]